In [1]:
import numpy as np
import xarray as xr
from odc.geo.xr import assign_crs, xr_reproject

from ocr import catalog
from ocr.risks.fire import apply_wind_directional_convolution, create_weighted_composite_bp_map
from ocr.utils import geo_sel

In [2]:
y_slice = slice(34.14, 34.23)
x_slice = slice(-118.18, -118.06)
buffer = 0.15

buffered_x_slice = slice(x_slice.start - buffer, x_slice.stop + buffer, x_slice.step)
buffered_y_slice = slice(y_slice.start - buffer, y_slice.stop + buffer, y_slice.step)
riley_2011_270m_5070 = catalog.get_dataset('riley-et-al-2025-2011-270m-5070').to_xarray()[
    ['BP', 'spatial_ref']
]
riley_2011_270m_5070 = assign_crs(riley_2011_270m_5070, 'EPSG:5070')

bbox = (
    buffered_x_slice.start,
    buffered_y_slice.start,
    buffered_x_slice.stop,
    buffered_y_slice.stop,
)
riley_2011_270m_5070_subset = geo_sel(
    riley_2011_270m_5070,
    bbox=bbox,
    crs_wkt=riley_2011_270m_5070.spatial_ref.attrs['crs_wkt'],
)
wind_direction_distribution_30m_4326 = (
    catalog.get_dataset('conus404-ffwi-p99-wind-direction-distribution-30m-4326')
    .to_xarray()
    .wind_direction_distribution.sel(latitude=buffered_y_slice, longitude=buffered_x_slice)
    .load()
)

In [3]:
def create_wind_informed_burn_probability(
    wind_direction_distribution_30m_4326: xr.DataArray,
    riley_270m_5070: xr.Dataset,
) -> xr.DataArray:
    """Create wind-informed burn probability dataset by applying directional convolution and creating a weighted composite burn probability map.

    Parameters
    ----------
    wind_direction_distribution_30m_4326 : xr.DataArray
        Wind direction distribution data at 30m resolution in EPSG:4326 projection.
    riley_270m_5070 : xr.DataArray
        Riley et al. (2011) burn probability data at 270m resolution in EPSG:5070 projection.

    Returns
    -------
    smoothed_final_bp : xr.DataArray
        Smoothed wind-informed burn probability data at 30m resolution in EPSG:4326 projection.
    """
    import cv2 as cv

    with xr.set_options(arithmetic_join='exact'):
        ## gap fill riley 270 only filling NaN pixels which are surrounded by valid data on four sides
        valid_pixels = riley_270m_5070['BP'] > 0
        # shifting will introduce NaNs! So we need to clip along boundaries
        surrounded_by_four_valid_values = (
            valid_pixels.shift(x=1)
            + valid_pixels.shift(x=-1)
            + valid_pixels.shift(y=1)
            + valid_pixels.shift(y=-1)
        ) == 4
        # 270m 5070 projection mask of every pixel which is surrounded by valid data on four sides
        nans_surrounded_by_four_valid_values = xr.where(
            (valid_pixels == 0) & surrounded_by_four_valid_values, 1, 0
        )
        # where nans_surrounded_by_four_valid_values is true, fill with a 3x3 moving window average of valid pixels
        rolling_mean = riley_270m_5070.rolling({'x': 3, 'y': 3}, center=True, min_periods=1).mean(
            skipna=True
        )
        gap_filled_riley_2011_270m_5070_subset = xr.where(
            nans_surrounded_by_four_valid_values, rolling_mean, riley_270m_5070
        )

        wind_direction_distribution_30m_4326 = assign_crs(
            wind_direction_distribution_30m_4326, 'EPSG:4326'
        )
        gap_filled_riley_2011_270m_5070_subset = assign_crs(
            gap_filled_riley_2011_270m_5070_subset, 'EPSG:5070'
        )
        ## reproject to the 30m 4326 projection (use riley_30m_4326 as variable name)
        target_geobox = wind_direction_distribution_30m_4326.odc.geobox
        riley_30m_4326 = xr_reproject(
            gap_filled_riley_2011_270m_5070_subset,
            how=target_geobox,
            resampling='nearest',
        )
        riley_30m_4326 = riley_30m_4326.assign_coords(
            {
                'latitude': wind_direction_distribution_30m_4326.latitude,
                'longitude': wind_direction_distribution_30m_4326.longitude,
            }
        )

        blurred_bp_30m_4326 = apply_wind_directional_convolution(riley_30m_4326['BP'], iterations=3)
        wind_informed_bp_30m_4326 = create_weighted_composite_bp_map(
            blurred_bp_30m_4326, wind_direction_distribution_30m_4326
        )
        wind_informed_bp_30m_4326 = assign_crs(wind_informed_bp_30m_4326, 'EPSG:4326')

        wind_informed_bp_30m_4326 = wind_informed_bp_30m_4326.assign_coords(
            {
                'latitude': wind_direction_distribution_30m_4326.latitude,
                'longitude': wind_direction_distribution_30m_4326.longitude,
            }
        )
        # gap fill any zeroes remaining in riley using the wind-smeared numbers

        # retain original Riley et al. (2025) burn probability where there are valid numbers at a 270m scale (but based upon
        # the dataset reprojected and interpolated to a 30m EPSG:4326 grid). anywhere there are no valid numbers use the
        # wind smeared values
        wind_informed_bp_combined = xr.where(
            riley_30m_4326['BP'] == 0, wind_informed_bp_30m_4326, riley_30m_4326['BP']
        )

    # smooth using a 21x21 Gaussian filter
    smoothed_bp = xr.apply_ufunc(
        cv.GaussianBlur,
        wind_informed_bp_combined.chunk(latitude=-1, longitude=-1),
        input_core_dims=[['latitude', 'longitude']],
        output_core_dims=[['latitude', 'longitude']],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[np.float32],
        kwargs={'ksize': (21, 21), 'sigmaX': 0},
    )
    smoothed_bp.name = 'BP'

    smoothed_bp.attrs = {
        'long_name': 'Wind-informed Burn Probability',
        'description': 'Wind-informed Burn Probability created by applying directional convolution and weighted composite using wind direction distribution',
    }
    return smoothed_bp, blurred_bp_30m_4326

In [4]:
%%time
smoothed_bp, blurred_bp_30m_4326 = create_wind_informed_burn_probability(
    wind_direction_distribution_30m_4326=wind_direction_distribution_30m_4326,
    riley_270m_5070=riley_2011_270m_5070_subset,
)
blurred_bp_30m_4326

/opt/coiled/env/lib/python3.13/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


CPU times: user 7.79 s, sys: 667 ms, total: 8.46 s
Wall time: 8.41 s


<xarray.Dataset> Size: 62MB
Dimensions:      (latitude: 1267, longitude: 1364)
Coordinates:
  * latitude     (latitude) float64 10kB 33.99 33.99 33.99 ... 34.38 34.38 34.38
  * longitude    (longitude) float64 11kB -118.3 -118.3 -118.3 ... -117.9 -117.9
    spatial_ref  int32 4B 4326
    crs          int64 8B 1
    quantile     float64 8B 0.99
Data variables:
    W            (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SW           (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    S            (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SE           (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    E            (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    NE           (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    N            (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    NW           (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    circular     (latitude, longitude) float32 7MB 0.0 0.0 0.0 ... 0.0 0.0 0.0